In [1]:
import os
import io
import csv
import base64
import detectron2
import glob
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# import some common libraries
import numpy as np
import cv2
import torch

import pandas as pd
from tqdm.notebook import tqdm
import json

# Show the image in ipynb
from IPython.display import clear_output, Image, display
import PIL.Image
def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [2]:
PATH = "/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged"
import pandas as pd
df = pd.read_csv(f"{PATH}/info.csv") 
df["image_id"] = df.index

In [3]:
df.head()

,Unnamed: 0,file_name,instruction,instruction_cluster,description,description_cluster,goal,image_id
0,0,./Town05/rgb/000000000000.jpg,turn to your two and go forward 5.0 meters,turn right walk,BUILDING on your left,BUILDING left,[3.5 3.5],0
1,1,./Town05/rgb/000000000000.jpg,walk forward 3.5 meters,walk,BUILDING on your left,BUILDING left,[-0.34 3.28],1
2,2,./Town05/rgb/000000000000.jpg,"turn to your one and afterwards, walk forward ...",turn right walk,BUILDING on your left,BUILDING left,[3.4 3.4],2
3,3,./Town05/rgb/000000000000.jpg,turn to your one and go forward 7.0 meters,turn right walk,BUILDING on your left,BUILDING left,[4.8 4.8],3
4,4,./Town05/rgb/000000000000.jpg,turn to your ten and walk straight 1.5 meters,turn left walk,BUILDING on your left,BUILDING left,[-0.98 1. ],4


In [4]:
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image

def doit(raw_image, NUM_OBJECTS):
    with torch.no_grad():
        raw_height, raw_width = raw_image.shape[:2]
        print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        print("Transformed image size: ", image.shape[:2])
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Generate proposals with RPN
        proposals, _ = predictor.model.proposal_generator(images, features, None)
        proposal = proposals[0]
        print('Proposal Boxes size:', proposal.proposal_boxes.tensor.shape)
        
        # Run RoI head for each proposal (RoI Pooling + Res5)
        proposal_boxes = [x.proposal_boxes for x in proposals]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        print('Pooled features size:', feature_pooled.shape)
        
        # Predict classes and boxes for each proposal.
        pred_class_logits, pred_attr_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        outputs = FastRCNNOutputs(
            predictor.model.roi_heads.box2box_transform,
            pred_class_logits,
            pred_proposal_deltas,
            proposals,
            predictor.model.roi_heads.smooth_l1_beta,
        )
        probs = outputs.predict_probs()[0]
        boxes = outputs.predict_boxes()[0]
        
        attr_prob = pred_attr_logits[..., :-1].softmax(-1)
        max_attr_prob, max_attr_label = attr_prob.max(-1)
        
        # Note: BUTD uses raw RoI predictions,
        #       we use the predicted boxes instead.
        # boxes = proposal_boxes[0].tensor    
        
        # NMS
        for nms_thresh in np.arange(0.5, 1.0, 0.1):
            instances, ids = fast_rcnn_inference_single_image(
                boxes, probs, image.shape[1:], 
                score_thresh=0.2, nms_thresh=nms_thresh, topk_per_image=NUM_OBJECTS
            )
            if len(ids) == NUM_OBJECTS:
                break
                
        instances = detector_postprocess(instances, raw_height, raw_width)
        roi_features = feature_pooled[ids].detach()
        max_attr_prob = max_attr_prob[ids].detach()
        max_attr_label = max_attr_label[ids].detach()
        instances.attr_scores = max_attr_prob
        instances.attr_classes = max_attr_label
        
        print(instances)
        
        return instances, roi_features


In [5]:
# Load VG Classes
data_path = 'data/genome/1600-400-20'

vg_classes = []
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        vg_classes.append(object.split(',')[0].lower().strip())

MetadataCatalog.get("vg").thing_classes = vg_classes


In [6]:
cfg = get_cfg()
cfg.merge_from_file("../configs/VG-Detection/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml")
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 300
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.MODEL.DEVICE = 'cuda:1'
# VG Weight
cfg.MODEL.WEIGHTS = "http://nlp.cs.unc.edu/models/faster_rcnn_from_caffe_attr.pkl"
predictor = DefaultPredictor(cfg)

Config '../configs/VG-Detection/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


Modifications for VG in ResNet Backbone (modeling/backbone/resnet.py):
	Using pad 0 in stem max_pool instead of pad 1.

Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/roi_heads.py):
	1. Change the stride of conv1 and shortcut in Res5.Block1 from 2 to 1.
	2. Modifying all conv2 with (padding: 1 --> 2) and (dilation: 1 --> 2).
	For more details, please check 'https://github.com/peteanderson80/bottom-up-attention/blob/master/models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'.

Modifications for VG in RoI heads (modeling/roi_heads/fast_rcnn.py))
	Embedding: 1601 --> 256	Linear: 2304 --> 512	Linear: 512 --> 401



In [7]:
def generate_feature(image, ids, num_objects=30):
    instances, features = doit(image, num_objects)

    boxes = instances.pred_boxes.tensor.cpu().numpy()
    image_height, image_width = instances.image_size
    feature = features.cpu().numpy()

    box_width = boxes[:, 2] - boxes[:, 0]
    box_height = boxes[:, 3] - boxes[:, 1]
    scaled_width = box_width/image_width
    scaled_height = box_height/image_height
    scaled_x = boxes[:, 0]/image_width
    scaled_y = boxes[:, 1] / image_height
    scaled_width = scaled_width[..., np.newaxis]
    scaled_height = scaled_height[..., np.newaxis]
    scaled_x = scaled_x[..., np.newaxis]
    scaled_y = scaled_y[..., np.newaxis]

    spatial_features = np.concatenate((scaled_x, scaled_y, scaled_x+scaled_width, scaled_y+scaled_height, scaled_width, scaled_height), axis=1)

    full_features = np.concatenate((feature, spatial_features), axis=1)
    fea_base64 = base64.b64encode(full_features).decode('utf-8')
    fea_info = {"num_boxes": boxes.shape[0], "features": fea_base64}

    objects = instances.pred_classes.cpu().numpy()
    conf = instances.scores.cpu().numpy()

    labels = []
    for i in range(len(boxes)):
        labels.append({
            "class": vg_classes[objects[i]],
            "rect": list(boxes[i, :]), 
            "conf": conf[i]
        })
    return {'image_id': ids, 'images': fea_info}, {'image_id': ids, 'list':labels}

In [8]:
def generate_feature_(rst, ids):
    return {'image_id': ids, 'images': rst[0]['images']}, {'image_id': ids, 'list':rst[1]['list']}

In [9]:
def generate_lineidx_file(filein, idxout):
    idxout_tmp = idxout + '.tmp'
    with open(filein, 'r') as tsvin, open(idxout_tmp,'w') as tsvout:
        fsize = os.fstat(tsvin.fileno()).st_size
        fpos = 0
        while fpos!=fsize:
            tsvout.write(str(fpos)+"\n")
            tsvin.readline()
            fpos = tsvin.tell()
    os.rename(idxout_tmp, idxout)

def generate_tsv(feature_path="./test/train.feature.tsv", label_path="./test/train.label.tsv"):
    with open(feature_path, 'a') as tsvfile, open(label_path, 'a') as labeltsvfile:
        writer = csv.DictWriter(tsvfile, delimiter='\t', fieldnames=['image_id', 'images'])
        Labelwriter=csv.DictWriter(labeltsvfile, delimiter='\t', fieldnames=['image_id', 'list'])
        img_tmp = ''
        for i in tqdm(range(df.shape[0])):
            img_path = PATH + f"{df.file_name.iloc[i][1:]}"
            print(img_path)
            if i%1000 == 0:
                print(img_path, img_tmp)
            if img_path == img_tmp:
                rst = generate_feature_(rst, df.image_id.iloc[i])
            else:
                im = cv2.imread(img_path)
                im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
                index = df.image_id.iloc[i]
                rst = generate_feature(im_rgb, ids=int(index))
                img_tmp = img_path

            writer.writerow(rst[0])
            Labelwriter.writerow(rst[1])
    generate_lineidx_file(feature_path, feature_path.replace("tsv", "lineidx"))
    generate_lineidx_file(label_path, label_path.replace("tsv", "lineidx"))



In [10]:
!rm ../test/train*.tsv

rm: cannot remove '../test/train*.tsv': No such file or directory


In [11]:
generate_tsv("../test/train.feature.tsv", "../test/train.label.tsv")

  0%|          | 0/30876 [00:00<?, ?it/s]

/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg 
Original image size:  (1080, 1920)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([122, 4])
Pooled features size: torch.Size([122, 2048])
Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000000.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000

Proposal Boxes size: torch.Size([137, 4])
Pooled features size: torch.Size([137, 2048])
Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000005.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town

Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000010.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Tow

Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000000016.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Tow

...

Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002039.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Tow

Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002045.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Tow

Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002051.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Tow

/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002056.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002056.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002056.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002057.jpg
Original image size:  (1080, 1920)
Transformed image size:  (750, 1333)
Proposal Boxes size: torch.Size([128, 4])
Pooled features size: torch.Size([128, 2048])
Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002057.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002057.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002057.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/0000000020

Instances(num_instances=30, image_height=1080, image_width=1920, fields=[pred_boxes, scores, pred_classes, attr_scores, attr_classes])
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Town05/rgb/000000002062.jpg
/data2/zhongkai/VIP/simulation_dataset/ECCV_test_stsw_merged/Tow